# NLP beginer

https://github.com/proger-n/DS_01_Tweets/blob/main/src/tweets.ipynb

### Introduction
NLP (natural language processing) — это область знаний и ряд методов и алгоритмов, которые помогают обрабатывать текстовые данные и извлекать из них информацию. Вы уже работали со структурированными данными — таблицами, полными непрерывных и категориальных признаков. Текст является примером полуструктурированных данных. С ним нужно что-то делать, чтобы использовать его, например, в задачах машинного обучения.

ЦЕЛЬ РАБОТЫ - предварительная обработка тектовых данных и обучение с помощью различных классификаторов для решения задачи классификации твитов на три класса (негативный, отрицательный и нейтральный)

### 1. Предварительная обработка текста:
Здесь перед подачей в классификатор необходимо предобработать текст, есть следующие способы:
- [Term Frequency-Inverse Document Frequency (TF-IDF)](https://habr.com/ru/companies/otus/articles/755772/) - это один из наиболее распространенных и мощных методов для извлечения признаков из текстовых данных. TF-IDF вычисляет важность каждого слова в документе относительно количества его употреблений в данном документе и во всей коллекции текстов. Этот метод позволяет выделить ключевые слова и понять, какие слова имеют больший вес для определенного документа в контексте всей коллекции.
Перед тем как вычислять TF-IDF, мы должны выполнить предварительную обработку, такую как удаление стоп-слов, приведение к нижнему регистру и токенизация — разбиение текстов на отдельные слова или токены.
- Стемминг (stemming) - это грубый эвристический процесс, который отрезает «лишнее» от корня слов, часто это приводит к потере словообразовательных суффиксов. Например слово "Коты", станет "Кот"
- Лематизация - это более тонкий процесс, который использует словарь и морфологический анализ, чтобы в итоге привести слово к его канонической форме – лемме.

Примеры лематизации и стемминга: Слово good – это лемма для слова better. Стеммер не увидит эту связь, так как здесь нужно сверяться со словарем.
Слово play – это базовая форма слова playing. Тут справятся и стемминг, и лемматизация.
Слово meeting может быть как нормальной формой существительного, так и формой глагола to meet, в зависимости от контекста.
В отличие от стемминга, лемматизация попробует выбрать правильную лемму,опираясь на контекст.

## 0.Import

In [1]:
import pandas as pd
import numpy as np
import zipfile
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from autocorrect import Speller
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
import warnings
warnings. filterwarnings('ignore')

nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

## 1. Подготовка данных
Данные хранятся в архиве. в Архиве три файла csv. В одном файле негативные отзыва, в другом позитивные и в третьем нейтральные. csv файлы содержат одну строку и много столбцов, поэтому нужно будет трансопнировать и привести всё в одну таблицу

In [2]:
# Распаковываем zip архив
with zipfile.ZipFile('../datasets/p00_tweets.zip', 'r') as zip_ref:
    zip_ref.extractall('data')
    
# Читаем файлы в pandas
positive_tweets = pd.read_csv('data/processedPositive.csv')
print(positive_tweets.shape)

neutral_tweets = pd.read_csv('data/processedNeutral.csv')
print(neutral_tweets.shape)

negative_tweets = pd.read_csv('data/processedNegative.csv')
print(negative_tweets.shape)

(0, 1186)
(0, 1570)
(0, 1117)


### 1.1. Приведем все данные в одну таблицу.
Для этого транспонируем исходные таблицы и назначим предсказательную переменную:
- 1 - негативный твит
- 2 - нейтральный твит
- 3 - положительный твит

In [3]:
def prepare_data(df, target_index):
    df_new = df.T.reset_index().rename(columns={'index': 'tweet'})
    df_new['target'] = target_index
    return df_new

In [4]:
negative_tweets = prepare_data(negative_tweets, target_index=1)
neutral_tweets = prepare_data(neutral_tweets, target_index=2)
positive_tweets = prepare_data(positive_tweets, target_index=3)

df = pd.concat([negative_tweets, neutral_tweets, positive_tweets], ignore_index=True)
df

,tweet,target
0,How unhappy some dogs like it though,1
1,talking to my over driver about where I'm goin...,1
2,Does anybody know if the Rand's likely to fall...,1
3,I miss going to gigs in Liverpool unhappy,1
4,There isnt a new Riverdale tonight ? unhappy,1
...,...,...
3868,Thanks for the recent follow Happy to connect ...,3
3869,- top engaged members this week happy,3
3870,ngam to weeks left for cadet pilot exam cryin...,3
3871,Great! You're welcome Josh happy ^Adam,3


### 1.2. чистим твиты
убираем все символы кроме букв и преобразуем к нижнему регистру

In [5]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r'[^a-zA-Z ]', '', regex=True)
    df[text_field] = df[text_field].str.lower()
    return df

df = standardize_text(df, 'tweet')
df

,tweet,target
0,how unhappy some dogs like it though,1
1,talking to my over driver about where im going...,1
2,does anybody know if the rands likely to fall ...,1
3,i miss going to gigs in liverpool unhappy,1
4,there isnt a new riverdale tonight unhappy,1
...,...,...
3868,thanks for the recent follow happy to connect ...,3
3869,top engaged members this week happy,3
3870,ngam to weeks left for cadet pilot exam cryin...,3
3871,great youre welcome josh happy adam,3


### 1.3. Токенизация
Разбиваем твиты на токены

И затем удалим строки где содержаться пустые твиты

In [6]:
tokenizer = RegexpTokenizer(r'\w+')

df['tokens'] = df['tweet'].apply(tokenizer.tokenize)
df

,tweet,target,tokens
0,how unhappy some dogs like it though,1,"[how, unhappy, some, dogs, like, it, though]"
1,talking to my over driver about where im going...,1,"[talking, to, my, over, driver, about, where, ..."
2,does anybody know if the rands likely to fall ...,1,"[does, anybody, know, if, the, rands, likely, ..."
3,i miss going to gigs in liverpool unhappy,1,"[i, miss, going, to, gigs, in, liverpool, unha..."
4,there isnt a new riverdale tonight unhappy,1,"[there, isnt, a, new, riverdale, tonight, unha..."
...,...,...,...
3868,thanks for the recent follow happy to connect ...,3,"[thanks, for, the, recent, follow, happy, to, ..."
3869,top engaged members this week happy,3,"[top, engaged, members, this, week, happy]"
3870,ngam to weeks left for cadet pilot exam cryin...,3,"[ngam, to, weeks, left, for, cadet, pilot, exa..."
3871,great youre welcome josh happy adam,3,"[great, youre, welcome, josh, happy, adam]"


In [7]:
def check_empty_list(cell):
    return cell == []

mask = df['tokens'].apply(check_empty_list)
df = df[~mask]
df

,tweet,target,tokens
0,how unhappy some dogs like it though,1,"[how, unhappy, some, dogs, like, it, though]"
1,talking to my over driver about where im going...,1,"[talking, to, my, over, driver, about, where, ..."
2,does anybody know if the rands likely to fall ...,1,"[does, anybody, know, if, the, rands, likely, ..."
3,i miss going to gigs in liverpool unhappy,1,"[i, miss, going, to, gigs, in, liverpool, unha..."
4,there isnt a new riverdale tonight unhappy,1,"[there, isnt, a, new, riverdale, tonight, unha..."
...,...,...,...
3868,thanks for the recent follow happy to connect ...,3,"[thanks, for, the, recent, follow, happy, to, ..."
3869,top engaged members this week happy,3,"[top, engaged, members, this, week, happy]"
3870,ngam to weeks left for cadet pilot exam cryin...,3,"[ngam, to, weeks, left, for, cadet, pilot, exa..."
3871,great youre welcome josh happy adam,3,"[great, youre, welcome, josh, happy, adam]"


In [8]:
all_words = [word for tokens in df['tokens'] for word in tokens] # Все слова во всех твитах
tweet_length = [len(tokens) for tokens in df['tokens']] # список с количеством слов в твите
different_words = sorted(list(set(all_words))) # список различных слов
print('Всего слов в твитах: ', len(all_words))
print('Всего твитов: ', len(df))
print('Длина максимального твита: ', max(tweet_length))
print('Длина минимального твита: ', min(tweet_length))
print('Различных слов во всех твитах: ', len(different_words))

Всего слов в твитах:  33208
Всего твитов:  3868
Длина максимального твита:  30
Длина минимального твита:  1
Различных слов во всех твитах:  6378


### 1.4. Функции для трансформорации слова в векторы с использованием различных подходов:
- бинаризация, переделка в onehot vector
- подсчет слов
- TFIDF
- Word2Vec

Все методы векторизации предназначены для перевода слов в вектора (числа)

#### 1.4.1. Бинаризация
про MultiLabelBinarizer() написано [тут](https://ru.stackoverflow.com/questions/928443/%D0%9A%D0%B0%D0%BA-%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D1%82-sklearn-preprocessing-multilabelbinarizer), [тут](https://spec-zone.ru/scikit_learn/modules/generated/sklearn.preprocessing.multilabelbinarizer) и [тут](https://scikit-learn.ru/1-12-multiclass-and-multioutput-algorithms/#multiclass-classification). В целом данный класс преобразует столбец с токенами в One-Hot-Encoded

In [9]:
def binarize_tokens_and_split(df, tokens_column_name, target_column_name, test_size=0.2):
    """
    Преобразует матрицу с токенами в one-hot-векторы и разбиват на тренировочную и тестовую
    :param df: датафрейм с токенами и целевой переменной
    :param tokens_column_name: название столбца датафрейма с токенами
    :param target_column_name: название столбца датафрейма с целевой переменной
    :param test_size: размер тестовой выборки
    :return: X_train_bin, X_test_bin, y_train, y_test
    """
    X_train, X_test, y_train, y_test = train_test_split(df[tokens_column_name],
                                                        df[target_column_name],
                                                        stratify=df[target_column_name],
                                                        test_size=test_size,
                                                        random_state=42)
    
    mlb = MultiLabelBinarizer()
    mlb.fit(X_train)  # получает классы - один класс - одно слово, где слова не повторяются
    X_train_bin = mlb.transform(X_train) # преобразуем в onehot vector
    X_test_bin = mlb.transform(X_test)  # преобразуем в onehot vector
    return X_train_bin, X_test_bin, y_train, y_test

#### 1.4.2. подсчет количества слов
про CountVectorizer() можно ознакомится [тута](https://github.com/Yorko/mlcourse.ai/blob/main/jupyter_russian/tutorials/vectorizers_tutorial_mvsamsonov.ipynb), [тута](https://habr.com/ru/articles/702626/) и [тут](https://habr.com/ru/articles/205360/). 
Если коротко то данный класс преобразует твиты в матрицу значениями которой, являются количества вхождения данного ключа(слова) в текст. Следует отметить что здесь нужно передать не список токенов, а список предложений. По умолчанию возвращается разреженная матрица

In [10]:
def count_words_and_split(df, tokens_column_name, target_column_name, test_size=0.2):
    """
    Преобразует матрицу с токенами сначала в матрице предложений,
    Затем заносит в матрицу для каждого слова - сколько раз оно встречалось во всей выборке
    Возвращает разреженную матрицу
    
    :param df: датафрейм с токенами и целевой переменной
    :param tokens_column_name: название столбца датафрейма с токенами
    :param target_column_name: название столбца датафрейма с целевой переменной
    :param test_size: размер тестовой выборки
    :return: X_train_counts, X_test_counts, y_train, y_test
    """
    list_corpus = df[tokens_column_name].apply(lambda x: ' '.join(x)).tolist() # из токенов получаем как бы список твитов в одном предложении
    list_labels = df[target_column_name].tolist() # преобразуем целевые переменные в список
    
    X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, stratify=list_labels, test_size=test_size, random_state=21)
    
    count_vectorizer = CountVectorizer()
    X_train_counts = count_vectorizer.fit_transform(X_train).toarray()
    X_test_counts = count_vectorizer.transform(X_test).toarray()
    
    return X_train_counts, X_test_counts, y_train, y_test

    

#### 1.4.3. TFIDF
про TFIDF написано [тут](https://github.com/Yorko/mlcourse.ai/blob/main/jupyter_russian/tutorials/vectorizers_tutorial_mvsamsonov.ipynb)
Если слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения CountVectorizer различий не будет. Здесь на помощь приходит TFIDF.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

In [11]:
def tfidf_and_split(df, tokens_column_name, target_column_name, test_size=0.2):
    """
    Преобазует матрицу с токенами в TFIDF
    
    :param df: датафрейм с токенами и целевой переменной
    :param tokens_column_name: название столбца датафрейма с токенами
    :param target_column_name: название столбца датафрейма с целевой переменной
    :param test_size: размер тестовой выборки
    :return: X_train_tfidf, X_test_tfidf, y_train, y_test
    """
    list_corpus = df[tokens_column_name].apply(lambda x: ' '.join(x)).tolist()
    list_labels = df[target_column_name].tolist()

    X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, stratify=list_labels, test_size=test_size, random_state=21)
    tfidf = TfidfVectorizer()
    tfidf.fit(X_train)
    X_train_tfidf = tfidf.transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    return X_train_tfidf, X_test_tfidf, y_train, y_test

#### 1.4.4. Word2Wec
Про то как работает Word2Vec можно посмотреть [тут](https://habr.com/ru/articles/585838/) и [тут](https://habr.com/ru/articles/446530/)

In [12]:
def word2vec_and_split(df: pd.DataFrame, tokens_column: str, target_column: str, vector_size: int = 80, min_count: int = 1,
                 window: int = 5, epochs: int = 80) -> pd.DataFrame:
    """The method return a "word2vec" column that contains the word2vec representation,
    text must be misspelled"""

    sentences = [text for text in df[tokens_column]]
    model = Word2Vec(sentences, vector_size=vector_size, min_count=min_count,
                     window=window, epochs=epochs)

    word2vec_vectors = []
    for text in sentences:
        if len(text) > 0:
            vector = sum(model.wv[word] for word in text if word in model.wv) / len(text)
        else:
            vector = [0] * vector_size
        word2vec_vectors.append(vector)

    # df["word2vec"] = word2vec_vectors
    # list_corpus = df["word2vec"].tolist()
    list_labels = df[target_column].tolist()

    X_train_w2v, X_test_w2v, y_train, y_test = train_test_split(word2vec_vectors, list_labels, stratify=list_labels, test_size=0.2, random_state=21)

    return X_train_w2v, X_test_w2v, y_train, y_test

### 1.5. Функции для препроцессинга слов перед векторизацией

In [13]:
def lemming(df, tokens_column_name):
    """
    Применяет лематизацию к токенам
    :param df: датафрейм
    :param tokens_column_name:  название столбца к которому нужно применить лематизацию
    :return: столбец с привденными леммами
    """
    lemmatizer = WordNetLemmatizer()
    return df[tokens_column_name].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [14]:
def stemming(df, tokens_column_name):
    """
    Применяет стемминг к токенам
    :param df: датафрейм
    :param tokens_column_name:  название столбца к которому нужно применить стемминг
    :return: столбец с привденными формами слов
    """
    stemmer = PorterStemmer()
    return df[tokens_column_name].apply(lambda x: [stemmer.stem(word) for word in x])

In [15]:
def stop_words_remove(df, tokens_column_name):
    """
    удаляет стоп слова
    :param df: датафрейм
    :param tokens_column_name:  название столбца в котором нужно удалить стоп слова
    :return: столбец с удаленными стоп словами
    """
    stop_words = set(stopwords.words('english'))
    return df[tokens_column_name].apply(lambda x: [word for word in x if word not in stop_words])

In [16]:
def misspelling(df, tokens_column_name):
    """
    Исправляет орфографические ошибки в словах
    :param df: датафрейм
    :param tokens_column_name:  название столбца в котором нужно исправить орфографические ошибки
    :return: столбец с исправлеными токенами
    """
    speller = Speller(lang="en")
    return df[tokens_column_name].apply(lambda words: [speller(word) for word in words])

### 1.6. Вспомогатльная функция для заполнения таблицы

In [17]:
def make_classification(prep_name, df, tokens_column, target_column, model):
    """
    
    :param prep_name: название преподготовки
    :param df: dataframe c признаками и целевой переменной
    :param tokens_column: название колнки с цлевой переменной
    :param target_column: название колонки с целевыми признаками (токенами)
    :param model: необученная модель с параметрами
    :return: точность предсказаний на тестовой выборке
    """
    result = [prep_name]
    X_train, X_test, y_train, y_test = binarize_tokens_and_split(df, tokens_column, target_column)
    result.append(accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test)))

    X_train, X_test, y_train, y_test = count_words_and_split(df, tokens_column, target_column)
    result.append(accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test)))

    X_train, X_test, y_train, y_test = tfidf_and_split(df, tokens_column, target_column)
    result.append(accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test)))
    
    X_train, X_test, y_train, y_test = word2vec_and_split(df, tokens_column, target_column)
    result.append(accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test)))

    return result

## 2. ML-алгоритмы
Поскольк нам дали возможность выбирать модели самим, будем пробовать четыре модели:
- логистическая регрессия
- дерево решений
- случайный лес
- метод опорных векторов

Так же попробуем использовать GridSeach для выбора оптиального параметра для каждо модели.
Поскольку есть много вариаций для которой можно применить GreadSearch, мы сначала применим его для TFIDF с предварительным препроцессингом датасета (стеминг лематизация и удаления стоп слов)
Затем выявим лучшие параметры и уже модель с этими параметрами применим ко всем вариациям согласно заданию и заполним таблицы для каждой модели.

### 2.0. Для наглядности как работают лематизация, стемминг, исправление ошибок и удаление стоп слов - создадим новый датафрейм
Посмотрим как выглядит столбец с токенами после каждой из данных процедур

In [18]:
df_temp = df[['target', 'tokens']]
df_temp['stemming'] = stemming(df_temp, 'tokens')
df_temp['lemming'] = lemming(df_temp, 'tokens')
df_temp['stemming_misspeling'] = misspelling(df_temp, 'stemming')
df_temp['lemming_misspeling'] = misspelling(df_temp, 'lemming')
df_temp['stopWords'] = stop_words_remove(df_temp, 'tokens')
df_temp['lemming_misspeling_stopWords'] = stop_words_remove(df_temp, 'lemming_misspeling')

df_temp

,target,tokens,stemming,lemming,stemming_misspeling,lemming_misspeling,stopWords,lemming_misspeling_stopWords
0,1,"[how, unhappy, some, dogs, like, it, though]","[how, unhappi, some, dog, like, it, though]","[how, unhappy, some, dog, like, it, though]","[how, unhappy, some, dog, like, it, though]","[how, unhappy, some, dog, like, it, though]","[unhappy, dogs, like, though]","[unhappy, dog, like, though]"
1,1,"[talking, to, my, over, driver, about, where, ...","[talk, to, my, over, driver, about, where, im,...","[talking, to, my, over, driver, about, where, ...","[talk, to, my, over, driver, about, where, im,...","[talking, to, my, over, driver, about, where, ...","[talking, driver, im, goinghe, said, hed, love...","[talking, driver, im, going, said, hed, love, ..."
2,1,"[does, anybody, know, if, the, rands, likely, ...","[doe, anybodi, know, if, the, rand, like, to, ...","[doe, anybody, know, if, the, rand, likely, to...","[doe, anybody, know, if, the, rand, like, to, ...","[doe, anybody, know, if, the, rand, likely, to...","[anybody, know, rands, likely, fall, dollar, g...","[doe, anybody, know, rand, likely, fall, dolla..."
3,1,"[i, miss, going, to, gigs, in, liverpool, unha...","[i, miss, go, to, gig, in, liverpool, unhappi]","[i, miss, going, to, gig, in, liverpool, unhappy]","[i, miss, go, to, gig, in, liverpool, unhappy]","[i, miss, going, to, gig, in, liverpool, unhappy]","[miss, going, gigs, liverpool, unhappy]","[miss, going, gig, liverpool, unhappy]"
4,1,"[there, isnt, a, new, riverdale, tonight, unha...","[there, isnt, a, new, riverdal, tonight, unhappi]","[there, isnt, a, new, riverdale, tonight, unha...","[there, isnt, a, new, reversal, tonight, unhappy]","[there, isnt, a, new, riverdale, tonight, unha...","[isnt, new, riverdale, tonight, unhappy]","[isnt, new, riverdale, tonight, unhappy]"
...,...,...,...,...,...,...,...,...
3868,3,"[thanks, for, the, recent, follow, happy, to, ...","[thank, for, the, recent, follow, happi, to, c...","[thanks, for, the, recent, follow, happy, to, ...","[thank, for, the, recent, follow, happy, to, c...","[thanks, for, the, recent, follow, happy, to, ...","[thanks, recent, follow, happy, connect, happy...","[thanks, recent, follow, happy, connect, happy..."
3869,3,"[top, engaged, members, this, week, happy]","[top, engag, member, thi, week, happi]","[top, engaged, member, this, week, happy]","[top, engage, member, thi, week, happy]","[top, engaged, member, this, week, happy]","[top, engaged, members, week, happy]","[top, engaged, member, week, happy]"
3870,3,"[ngam, to, weeks, left, for, cadet, pilot, exa...","[ngam, to, week, left, for, cadet, pilot, exam...","[ngam, to, week, left, for, cadet, pilot, exam...","[nam, to, week, left, for, cadet, pilot, exam,...","[nam, to, week, left, for, cadet, pilot, exam,...","[ngam, weeks, left, cadet, pilot, exam, crying...","[nam, week, left, cadet, pilot, exam, cry, joy]"
3871,3,"[great, youre, welcome, josh, happy, adam]","[great, your, welcom, josh, happi, adam]","[great, youre, welcome, josh, happy, adam]","[great, your, welcome, josh, happy, adam]","[great, youre, welcome, josh, happy, adam]","[great, youre, welcome, josh, happy, adam]","[great, youre, welcome, josh, happy, adam]"


Для того что бы ориентировочно прикинуть лучшие параметры для четырех алгоритмов (логистическая регрессия, дерево решений, случайный лес, метод опорных векторов)
выберем примерно лучшие параметры для каждого алгоритма с использованием GreadSearchCV.
В качестве преобразователя слов в вектора возмем TFIDF.
А в качестве признаков возмем твиты после лематизации + исправление ошибок (misspelling) + удаление стоп-слов

In [19]:
X_train_tf, X_test_tf, y_train, y_test = tfidf_and_split(df_temp, target_column_name='target', tokens_column_name='lemming_misspeling_stopWords')

### 2.0.1. Для начала предварительно выберем "наилучшие" параметры для каждой модели

In [20]:
# 1. ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ
clf = LogisticRegression()
param_grid = {'C': [0.1, 1, 10, 30, 50],
              'solver': ['newton-cg', 'saga'],
              'penalty': ['l2', 'l1'],
              'max_iter': [3000],
              'random_state': [21],
              'class_weight': ['balanced'],
              'multi_class': ['multinomial'],
              'n_jobs': [-1]
              }
gs = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'C': 1, 'class_weight': 'balanced', 'max_iter': 3000, 'multi_class': 'multinomial', 'n_jobs': -1, 'penalty': 'l2', 'random_state': 21, 'solver': 'saga'}
лучший score: 0.879118109906886


In [21]:
# 2. ДЕРЕВО РЕШЕНИЙ
tree = DecisionTreeClassifier()
param_grid = {'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [21]
            }
gs = GridSearchCV(tree, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': None, 'criterion': 'gini', 'max_depth': 48, 'random_state': 21}
лучший score: 0.8648953578953421


In [22]:
# 3. Случайный лес
forest = RandomForestClassifier()
param_grid = {'n_estimators': [5, 10, 50, 100, 200, 300],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50, 2),
              'class_weight': ['balanced', None],
              'random_state': [21]
              }
 
gs = GridSearchCV(forest, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 41, 'n_estimators': 200, 'random_state': 21}
лучший score: 0.8668318772840629


In [23]:
svc = SVC()
param_grid = {'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None],
              'random_state': [21]
              }
gs = GridSearchCV(svc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'C': 1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'linear', 'random_state': 21}
лучший score: 0.8794417345023552


Теперь когда мы получили примерно оптимальные гипперпараметры моежм приступить к следующему
В следующем шаге мы будем использовать именно "лучшие" параметры для каждой модели

### 2.1. Логистическая регрессия

In [24]:
%%time

clf = LogisticRegression(C=1,
                         class_weight='balanced',
                         max_iter=3000,
                         solver='saga',
                         multi_class='multinomial',
                         n_jobs=-1,
                         random_state=21,
                         penalty='l2',)

# Создаем датафрейм кудла будем заносить метрики
results_logreg = pd.DataFrame({'preprocces':[], '0 or 1 if the word exists': [], 'word counts':[], 'TFIDF':[], 'Word2Vec':[]})

# Заполняем табличку
results_logreg.loc[len(results_logreg)] = make_classification('just tokenization', df_temp, 'tokens', 'target', clf)
print("1 из 7. Расчет точности без препроцессинга завершен")

results_logreg.loc[len(results_logreg)] = make_classification('stemming', df_temp, 'stemming', 'target', clf)
print("2 из 7. Расчет точности после стемминга завершен")

results_logreg.loc[len(results_logreg)] = make_classification('lemmatization', df_temp, 'lemming', 'target', clf)
print("3 из 7. Расчет точности после лематизации завершен")

results_logreg.loc[len(results_logreg)] = make_classification('stemming+misspellings', df_temp, 'stemming_misspeling', 'target', clf)
print("4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен")

results_logreg.loc[len(results_logreg)] = make_classification('lemmatization+misspellings', df_temp, 'lemming_misspeling', 'target', clf)
print("5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен")

results_logreg.loc[len(results_logreg)] = make_classification('stopWords', df_temp, 'stopWords', 'target', clf)
print("6 из 7. Расчет точности после удаления стоп-слов завершен")

results_logreg.loc[len(results_logreg)] = make_classification('lemmatization+misspellings+stopWords', df_temp, 'lemming_misspeling_stopWords', 'target', clf)
print("7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен")

1 из 7. Расчет точности без препроцессинга завершен
2 из 7. Расчет точности после стемминга завершен
3 из 7. Расчет точности после лематизации завершен
4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен
5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен
6 из 7. Расчет точности после удаления стоп-слов завершен
7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен
CPU times: user 2min 52s, sys: 13.5 s, total: 3min 6s
Wall time: 2min 44s


In [25]:
results_logreg

,preprocces,0 or 1 if the word exists,word counts,TFIDF,Word2Vec
0,just tokenization,0.910853,0.887597,0.886305,0.878553
1,stemming,0.909561,0.894057,0.890181,0.856589
2,lemmatization,0.905685,0.888889,0.886305,0.872093
3,stemming+misspellings,0.909561,0.892765,0.891473,0.855297
4,lemmatization+misspellings,0.906977,0.887597,0.885013,0.881137
5,stopWords,0.886305,0.887597,0.885013,0.875969
6,lemmatization+misspellings+stopWords,0.886305,0.879845,0.872093,0.854005


### 2.2. Дерево решений

In [26]:
%%time

tree = DecisionTreeClassifier(class_weight=None,
                              criterion='gini',
                              max_depth=48,
                              random_state=21)

# Создаем датафрейм кудла будем заносить метрики
results_tree = pd.DataFrame({'preprocces':[], '0 or 1 if the word exists': [], 'word counts':[], 'TFIDF':[], 'Word2Vec':[]})

# Заполняем табличку
results_tree.loc[len(results_tree)] = make_classification('just tokenization', df_temp, 'tokens', 'target', tree)
print("1 из 7. Расчет точности без препроцессинга завершен")

results_tree.loc[len(results_tree)] = make_classification('stemming', df_temp, 'stemming', 'target', tree)
print("2 из 7. Расчет точности после стемминга завершен")

results_tree.loc[len(results_tree)] = make_classification('lemmatization', df_temp, 'lemming', 'target', tree)
print("3 из 7. Расчет точности после лематизации завершен")

results_tree.loc[len(results_tree)] = make_classification('stemming+misspellings', df_temp, 'stemming_misspeling', 'target', tree)
print("4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен")

results_tree.loc[len(results_tree)] = make_classification('lemmatization+misspellings', df_temp, 'lemming_misspeling', 'target', tree)
print("5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен")

results_tree.loc[len(results_tree)] = make_classification('stopWords', df_temp, 'stopWords', 'target', tree)
print("6 из 7. Расчет точности после удаления стоп-слов завершен")

results_tree.loc[len(results_tree)] = make_classification('lemmatization+misspellings+stopWords', df_temp, 'lemming_misspeling_stopWords', 'target', tree)
print("7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен")

1 из 7. Расчет точности без препроцессинга завершен
2 из 7. Расчет точности после стемминга завершен
3 из 7. Расчет точности после лематизации завершен
4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен
5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен
6 из 7. Расчет точности после удаления стоп-слов завершен
7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен
CPU times: user 25.8 s, sys: 460 ms, total: 26.3 s
Wall time: 21.3 s


In [27]:
results_tree

,preprocces,0 or 1 if the word exists,word counts,TFIDF,Word2Vec
0,just tokenization,0.894057,0.874677,0.855297,0.771318
1,stemming,0.881137,0.874677,0.850129,0.757106
2,lemmatization,0.892765,0.875969,0.865633,0.748062
3,stemming+misspellings,0.891473,0.861757,0.855297,0.731266
4,lemmatization+misspellings,0.887597,0.869509,0.861757,0.733850
5,stopWords,0.873385,0.870801,0.874677,0.815245
6,lemmatization+misspellings+stopWords,0.874677,0.866925,0.868217,0.781654


### 2.3. Случайный лес

In [28]:
%%time

forest = RandomForestClassifier(class_weight='balanced',
                                criterion='gini',
                                max_depth=41,
                                random_state=21,
                                n_estimators=200,)

# Создаем датафрейм кудла будем заносить метрики
results_forest = pd.DataFrame({'preprocces':[], '0 or 1 if the word exists': [], 'word counts':[], 'TFIDF':[], 'Word2Vec':[]})

# Заполняем табличку
results_forest.loc[len(results_forest)] = make_classification('just tokenization', df_temp, 'tokens', 'target', forest)
print("1 из 7. Расчет точности без препроцессинга завершен")

results_forest.loc[len(results_forest)] = make_classification('stemming', df_temp, 'stemming', 'target', forest)
print("2 из 7. Расчет точности после стемминга завершен")

results_forest.loc[len(results_forest)] = make_classification('lemmatization', df_temp, 'lemming', 'target', forest)
print("3 из 7. Расчет точности после лематизации завершен")

results_forest.loc[len(results_forest)] = make_classification('stemming+misspellings', df_temp, 'stemming_misspeling', 'target', forest)
print("4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен")

results_forest.loc[len(results_forest)] = make_classification('lemmatization+misspellings', df_temp, 'lemming_misspeling', 'target', forest)
print("5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен")

results_forest.loc[len(results_forest)] = make_classification('stopWords', df_temp, 'stopWords', 'target', forest)
print("6 из 7. Расчет точности после удаления стоп-слов завершен")

results_forest.loc[len(results_forest)] = make_classification('lemmatization+misspellings+stopWords', df_temp, 'lemming_misspeling_stopWords', 'target', forest)
print("7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен")

1 из 7. Расчет точности без препроцессинга завершен
2 из 7. Расчет точности после стемминга завершен
3 из 7. Расчет точности после лематизации завершен
4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен
5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен
6 из 7. Расчет точности после удаления стоп-слов завершен
7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен
CPU times: user 1min 33s, sys: 530 ms, total: 1min 33s
Wall time: 1min 29s


In [29]:
results_forest

,preprocces,0 or 1 if the word exists,word counts,TFIDF,Word2Vec
0,just tokenization,0.904393,0.879845,0.874677,0.864341
1,stemming,0.903101,0.882429,0.881137,0.859173
2,lemmatization,0.900517,0.883721,0.879845,0.848837
3,stemming+misspellings,0.905685,0.878553,0.881137,0.846253
4,lemmatization+misspellings,0.905685,0.877261,0.873385,0.847545
5,stopWords,0.870801,0.869509,0.873385,0.866925
6,lemmatization+misspellings+stopWords,0.874677,0.870801,0.868217,0.865633


### 2.4. Метод опорных векторов

In [30]:
%%time

svc = SVC(C=1,
          class_weight=None,
          gamma='scale',
          kernel='linear',
          random_state=21)

# Создаем датафрейм кудла будем заносить метрики
results_svc = pd.DataFrame({'preprocces':[], '0 or 1 if the word exists': [], 'word counts':[], 'TFIDF':[], 'Word2Vec':[]})

# Заполняем табличку
results_svc.loc[len(results_svc)] = make_classification('just tokenization', df_temp, 'tokens', 'target', svc)
print("1 из 7. Расчет точности без препроцессинга завершен")

results_svc.loc[len(results_svc)] = make_classification('stemming', df_temp, 'stemming', 'target', svc)
print("2 из 7. Расчет точности после стемминга завершен")

results_svc.loc[len(results_svc)] = make_classification('lemmatization', df_temp, 'lemming', 'target', svc)
print("3 из 7. Расчет точности после лематизации завершен")

results_svc.loc[len(results_svc)] = make_classification('stemming+misspellings', df_temp, 'stemming_misspeling', 'target', svc)
print("4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен")

results_svc.loc[len(results_svc)] = make_classification('lemmatization+misspellings', df_temp, 'lemming_misspeling', 'target', svc)
print("5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен")

results_svc.loc[len(results_svc)] = make_classification('stopWords', df_temp, 'stopWords', 'target', svc)
print("6 из 7. Расчет точности после удаления стоп-слов завершен")

results_svc.loc[len(results_svc)] = make_classification('lemmatization+misspellings+stopWords', df_temp, 'lemming_misspeling_stopWords', 'target', svc)
print("7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен")

1 из 7. Расчет точности без препроцессинга завершен
2 из 7. Расчет точности после стемминга завершен
3 из 7. Расчет точности после лематизации завершен
4 из 7. Расчет точности после исправления орфографических ошибок и стеминга завершен
5 из 7. Расчет точности после исправления орфографических ошибок и лематизации завершен
6 из 7. Расчет точности после удаления стоп-слов завершен
7 из 7. Расчет точности после лематизации, исправления ошибок и удаления стоп-слов завершен
CPU times: user 2min 10s, sys: 946 ms, total: 2min 11s
Wall time: 2min 6s


In [31]:
results_svc

,preprocces,0 or 1 if the word exists,word counts,TFIDF,Word2Vec
0,just tokenization,0.897933,0.883721,0.883721,0.870801
1,stemming,0.901809,0.888889,0.891473,0.856589
2,lemmatization,0.897933,0.881137,0.888889,0.864341
3,stemming+misspellings,0.895349,0.881137,0.887597,0.864341
4,lemmatization+misspellings,0.900517,0.882429,0.887597,0.869509
5,stopWords,0.873385,0.882429,0.887597,0.868217
6,lemmatization+misspellings+stopWords,0.878553,0.878553,0.879845,0.857881


## 3. Выводы

#### Лучшие результаты для алгоритмов машиного обучения:

| алгоритм обучения | лучшая точность | параметры |
|---------------|-----------------|-----------|
| логистическая регрессия | 0.9109 | только токенизация и OneHot-вектор |
| дерево решений | 0.8941 | только токенизация и OneHot-вектор |
| случайный лес | 0.9056 | лематизация+misspeling и OneHot-вектор |
| метод опорных векторов | 0.9019 | стеминг и OneHot-вектор |

#### Лучшие результаты для алгоритмов перевода слов в вектора:

| алгоритм векторизации                                               | лучшая точность                          | параметры                                      |
|---------------------------------------------------------------------|------------------------------------------|------------------------------------------------|
| OneHot-вектор | 0.9109                                   | только токенизация и логистическая регрессия   |  
| Подсчет слов | 0.8941 | стеминг и логистическая регрессия              |             
| TFIDF | 0.8915 | стеминг и метод опорных векторов               | 
| Word2Vec | 0.8773 | только токенизация и логистичесская регрессия  | 